In [1]:
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from shapely import  Point
from itertools import combinations
sys.path.append(os.path.abspath('../raw_to_map'))

from im_to_im import epipolar_line, load_buildings, match_points, get_overlap, find_duplicate_indices_and_remove


In [2]:
with open('../AgderOst/settings.json') as f:
    settings = json.load(f)
with open('overlap.json') as f:
    overlap = json.load(f)
buildings = load_buildings('../AgderOst/buildings_transformed')

  0%|          | 0/8465 [00:00<?, ?it/s]

100%|██████████| 8465/8465 [00:04<00:00, 1836.99it/s]

Loaded 8465 building objects from ../AgderOst/buildings_transformed


In [20]:
def get_true_matches(iid1, iid2, buildings):
    true_matches = dict()
    for building in buildings:
        iids = building['corners'].keys()

        if iid1 in iids and iid2 in iids:
            p1 = Point(np.round(np.mean(np.array(list(building['corners'][iid1])), axis=0), 6))
            p2 = Point(np.round(np.mean(np.array(list(building['corners'][iid2])), axis=0), 6))
            true_matches[p1] = p2
    return true_matches

In [ ]:
iid1 = '14583_01_012_20028'
buildings_pt = {iid_adj: set() for iid_adj in overlap[iid1]} 
buildings_pt[iid1] = set()
iid2 = list(buildings_pt.keys())[7]
iid3 = list(buildings_pt.keys())[1]
true_matches = dict()
for building in buildings:
    iids = set(building['corners'].keys()).intersection(buildings_pt.keys())
    if iids:
        for iid in iids:
            point = Point(np.round(np.mean(np.array(list(building['corners'][iid])), axis=0), 6))
            buildings_pt[iid].add(point)
    

[2601.80792091 7715.96879735] [2601.807921 7715.968797]
[5953.98857924 7712.01735408] [5953.988579 7712.017354]
[ 2130.09784446 11552.08092718] [ 2130.097844 11552.080927]
[ 5447.17611426 11540.44165605] [ 5447.176114 11540.441656]
[1574.76953392 6471.21804168] [1574.769534 6471.218042]
[4963.79142814 6464.1156567 ] [4963.791428 6464.115657]
[  225.71674161 11375.4452053 ] [  225.716742 11375.445205]
[ 3512.52317364 11354.32402266] [ 3512.523174 11354.324023]
[15513.09455277 21614.34870865] [15513.094553 21614.348709]
[  581.61846927 12651.06791414] [  581.618469 12651.067914]
[ 3860.15215692 12629.96111723] [ 3860.152157 12629.961117]
[2779.32271669 2616.46598334] [2779.322717 2616.465983]
[6235.34635198 2620.9935898 ] [6235.346352 2620.99359 ]
[3485.77222149 8897.6568368 ] [3485.772221 8897.656837]
[  552.04629107 13581.44376929] [  552.046291 13581.443769]
[ 3812.0079459  13558.82322116] [ 3812.007946 13558.823221]
[ 194.70106963 8171.20752042] [ 194.70107 8171.20752]
[3571.13926139

In [34]:

iid1 = list(buildings_pt.keys())[4]
iid2 = list(buildings_pt.keys())[1]
true_matches = get_true_matches(iid1, iid2, buildings)
matches, mult_match = match_points(buildings_pt[iid1], buildings_pt[iid2], iid1, iid2)

counter = 0
for m in matches:
    if matches[m] == true_matches[m]:
        counter += 1
mult_tot = sum([len(m) for m in mult_match])
print (f'{counter}/{len(matches)} -- {round(counter/len(matches), 4)} -- N matches: {len(matches)} -- N true matches: {len(true_matches)} -- mult match:{mult_tot}\n')




469/469 -- 1.0 -- N matches: 469 -- N true matches: 469 -- mult match:0



In [23]:
keys = list()
for m in mult_match:
    print (m, mult_match[m])

(<POINT (2536.549 7269.31)>, <POINT (595.739 7260.063)>) POINT (9282.161484 7304.411061)
(<POINT (9067.201 19015.197)>, <POINT (10212.57 19017.91)>) POINT (16791.668424 19050.545483)


In [32]:
def unique_shapely(points, tol=1e-2):
    unique = []
    
    for point in points:
        # Check if the point is close to any point in the unique list
        if not any(point.distance(other) <= tol for other in unique):
            unique.append(point)
    
    return unique